In [1]:
import matplotlib.pyplot as plt
import torch
from Net import Net
from Data import Data
from Train import train_free_energy, fit, train_free_energy_two_inputs
import numpy as np
from MyLossFunction import free_energy
from scipy import integrate
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
torch.manual_seed(3)
%matplotlib qt

#### Make some data

In [2]:
points = 499
lower = 0.00001
upper = 2*np.pi + lower
data_set = Data(points, lower, upper)
x, y = data_set.get()
matrix = data_set.matrix()
simpson_matrix = data_set.simpson_matrix()
input_var = data_set.input_variables()

#### Implement neural network, optimizer and scheduler，with learning rate decay

In [6]:
Layers = [2, 16, 16, 1]
net = Net(Layers)
learning_rate = 0.02
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.9)

#### An example

In [9]:
rho = 4.5
lambd = 200
net = train_free_energy_two_inputs(net, x, input_var, rho, lambd, scheduler,
                                   simpson_matrix ,optimizer, matrix, epochs = 10000, diagram=True)
output = net(input_var)

In [15]:
free_energy(output, rho, x, simpson_matrix, matrix, lambd)

tensor([[0.2118]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [30]:
plt.plot(x.data.numpy(), output.data.numpy())
plt.ylim([0, 0.6])
plt.show()

In [9]:
Layers = [1, 16, 1]
learning_rate = 0.02
net = Net(Layers)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.9)
rho = 3
lambd = 200
net = train_free_energy(net, x, rho, lambd,
                                   optimizer, matrix, epochs=10000, diagram=True)

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #2 'mat1' in call to _th_addmm

In [3]:
## normalize
factor = torch.trapz(output.T,x.T)
print ("%.10f"%factor)
output = output/factor
plt.plot(x.data.numpy(), output.data.numpy())
print (torch.trapz(output.T,x.T))
print (torch.max(output))

NameError: name 'torch' is not defined

In [60]:
## calculate the loss
I = free_energy(output, x, matrix, rho, lambd)
print ("the number of points is:", points)
print ("the loss of real output is %.8f"%I)
output.data.numpy()[0],output.data.numpy()[len(x)-1] = output.data.numpy()[2], output.data.numpy()[2]
plt.plot(x.data.numpy(), output.data.numpy())
I_2 = free_energy(output, x, matrix, rho, lambd)
print ("the loss of modified (what we want) output is %.8f"%I_2)

the number of points is: 256
the loss of real output is 1.10909365
the loss of modified (what we want) output is 1.10909365


In [19]:
def free_energy_numpy(yhat, x, matrix, rho, lambd):
    L = len(yhat)
    first_term = integrate.trapz((yhat*np.log(rho*yhat)).T, x.T)  #todo： this should be right
    second_term = (2 * np.pi ** 2 * rho / (L-1) ** 2) * np.dot(np.dot(yhat.T, matrix), yhat)  # todo: this part is wrong
    third_term = lambd * np.square(integrate.trapz(yhat.T, x.T) - 1)  # the only reason we need x
    loss = first_term \
           + second_term \
           + third_term
    return loss

In [21]:
output = net(input_var)
output = output.data.numpy()
I = free_energy_numpy(output, x, matrix, rho, lambd)
print ("the number of points is:", points)
print ("the loss of real output is %.8f"%I)
output_first = output[0: 250]
output_second = output[250: 999]
output_final = np.append(output_second, output_first)
I_2 = free_energy_numpy(output_final, x, matrix, rho, lambd)
print ("the loss of modified (what we want) output is %.8f"%I_2)
plt.plot(x.data.numpy(), output_final)

the number of points is: 1000
the loss of real output is 1.46444726
the loss of modified (what we want) output is 1.47021735


In [ ]:
# calculate the loss

#### Pre-train can make the cost curve more obvious

In [7]:
# net_dict_prev = {}
# rho_list = np.linspace(4, 6, 10)
# for i in range (len(rho_list)):
#     rho = rho_list[i]
#     net = Net(Layers)
#     learning_rate = 0.015
#     optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.9)
#     lambd = 25
#     net_dict_prev["rho=" + str(i)] = train_free_energy_two_inputs(net, x,  input_var, rho, lambd, scheduler,
#                                        optimizer, matrix, epochs=500, diagram=False)

In [4]:
Layers = [2, 16, 16, 16, 16, 16, 16, 1]
rho_list = np.linspace(4.2, 5.7, 7)
net_dict = {}
output_list = {}
i = 0
for rho in rho_list:
    net = Net(Layers)
    learning_rate = 0.013
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.9)
    lambd = 25
    net_dict["rho=" + str(i)] = train_free_energy_two_inputs(net, x,  input_var, rho, lambd, scheduler,
                                       optimizer, matrix, epochs=7000, diagram=False)
    output_list["rho=" + str(i)] = net_dict["rho=" + str(i)](input_var).data.numpy()
    i+= 1

In [12]:
# normalization:
# i = 0
# for rho in rho_list:
#     I = torch.trapz(output_list["rho=" + str(i)].T, x.T)
#     output_list["rho=" + str(i)] = output_list["rho=" + str(i)]/I
#     i+=1

TypeError: trapz() received an invalid combination of arguments - got (numpy.ndarray, Tensor), but expected one of:
 * (Tensor y, *, float dx, int dim)
 * (Tensor y, Tensor x, *, int dim)


In [5]:
#这太蠢了，有没有简洁的写法？？？
height = np.concatenate([output_list["rho=" + str(0)],
                         output_list["rho=" + str(1)],
                         output_list["rho=" + str(2)],
                         output_list["rho=" + str(3)],
                         output_list["rho=" + str(4)],
                         output_list["rho=" + str(5)],
                         output_list["rho=" + str(6)]],
                        axis= 1)

In [6]:
fig3d = plt.figure()
ax = Axes3D(fig3d)
X = np.linspace(0.000001, 2 * np.pi, 256)
rho_list, X = np.meshgrid(rho_list, X)    # x-y grid
ax.plot_surface(rho_list, X , height, rstride=1, cstride=1, cmap=plt.get_cmap('rainbow'))

In [18]:
print (rho_list[1:13])

[[4.         4.14285714 4.28571429 ... 5.71428571 5.85714286 6.        ]
 [4.         4.14285714 4.28571429 ... 5.71428571 5.85714286 6.        ]
 [4.         4.14285714 4.28571429 ... 5.71428571 5.85714286 6.        ]
 ...
 [4.         4.14285714 4.28571429 ... 5.71428571 5.85714286 6.        ]
 [4.         4.14285714 4.28571429 ... 5.71428571 5.85714286 6.        ]
 [4.         4.14285714 4.28571429 ... 5.71428571 5.85714286 6.        ]]
